

### **1. Luokat ja niiden toiminnot**

#### **`Action`**
- **Mikä se on?**
  - Tämä luokka kuvaa yksittäistä toimintoa (syöminen, nukkuminen, työnteko).
  - Jokaisella toiminnolla on energiaan ja rahaan liittyviä vaikutuksia.
- **Keskeiset ominaisuudet:**
  - `name`: Toiminnon nimi (esim. "Eat", "Sleep", "Work").
  - `energy_change`: Kuinka paljon toiminto lisää tai vähentää energiaa.
  - `money_change`: Kuinka paljon toiminto lisää tai vähentää rahaa.
- **Metodi `execute`:**
  - Päivittää nykyisen tilan (energia ja raha) toiminnon vaikutusten mukaan.

#### **`Planner`**
- **Mikä se on?**
  - Tämä luokka hallinnoi toimintojen suunnittelua ja varmistaa, että säännöt ja rajoitukset täyttyvät.
- **Keskeiset ominaisuudet:**
  - `actions`: Lista mahdollisista toiminnoista.
  - `goal`: Rahamäärä, joka pitää saavuttaa (esim. 1000).
  - `max_consecutive_actions`: Maksimimäärä peräkkäisiä samanlaisia toimintoja.
  - `max_energy`: Suurin sallittu energiataso (esim. 300).
  - `max_wakefulness`: Kuinka monta toimintoa voi tehdä ennen kuin on pakko nukkua.
  - `wakefulness`: Laskee, montako toimintoa on tehty ilman nukkumista.

---

### **2. Toimintojen validointi (`is_valid`)**
Tämä metodi tarkistaa, onko tietty toiminto sallittu nykyisessä tilassa. Se käyttää seuraavia ehtoja:

1. **Energiarajoitus:**
   - Toiminnon jälkeen energiataso ei saa olla alle 0 tai yli maksimin.
   ```python
   new_energy = state['energy'] + action.energy_change - 50
   if new_energy <= 0 or new_energy > self.max_energy:
       return False
   ```

2. **Rahan rajoitus:**
   - Rahamäärä ei saa mennä alle 0 toiminnon jälkeen.
   ```python
   if state['money'] + action.money_change <= 0:
       return False
   ```

3. **Hereilläolon rajoitus:**
   - Jos `wakefulness` ylittää sallitun maksimin, ainoa sallittu toiminto on nukkuminen.
   ```python
   if self.wakefulness >= self.max_wakefulness and action.name != "Sleep":
       return False
   ```

4. **Peräkkäisten toimintojen rajoitus:**
   - Sama toiminto ei saa toistua 3 kertaa peräkkäin.
   ```python
   if len(action_history) >= self.max_consecutive_actions - 1:
       if all(a == action.name for a in action_history[-(self.max_consecutive_actions - 1):]):
           return False
   ```

---

### **3. Suunnittelun logiikka (`plan`)**

1. **Tavoitteen asettaminen:**
   - Tulostaa tavoitteen ja alkutilan.
   ```python
   print(f"Goal: Earn at least {self.goal} money while managing energy and avoiding restrictions.")
   print(f"Initial State: {state}\n")
   ```

2. **Toimintojen suorittaminen:**
   - Suorittaa toimintoja, kunnes tavoite (rahaa vähintään `goal`) saavutetaan.
   - Jokaisen validin toiminnon jälkeen:
     - Päivittää energiaa (-50 yleiskustannus).
     - Suorittaa toiminnon vaikutukset (`execute`).
     - Lisää toiminnon historiaan.
     - Päivittää hereilläololaskurin:
       - Nukkuminen nollaa laskurin.
       - Muut toiminnot kasvattavat laskuria.

3. **Tulostus suorituksen aikana:**
   - Tulostaa jokaisen tehdyn toiminnon ja sen jälkeisen tilan.
   ```python
   print(f"Performed: {action.name}, State: {state}")
   ```

4. **Lopputuloksen tulostus:**
   - Tavoitteen saavuttamisen jälkeen ohjelma tulostaa lopullisen tilan ja toimintojen järjestyksen.
   ```python
   print("\nFinal State:")
   print(final_state)
   print("\nAction Sequence:")
   print(action_sequence)
   ```

---

### **4. Pääohjelma**

1. **Toimintojen määrittely:**
   - "Eat": Lisää 200 energiaa, maksaa 50 rahaa.
   - "Sleep": Lisää 100 energiaa, ei maksa mitään.
   - "Work": Lisää 100 rahaa, vähentää 100 energiaa.

2. **Alkutila:**
   - Energia: 200.
   - Raha: 50.

3. **Suunnittelijan luonti:**
   - Luodaan `Planner`, jossa on määritetty hereilläolon maksimi (esim. 5 toimintoa ennen nukkumista).

4. **Suunnitelman suorittaminen:**
   - Kutsutaan `plan`-metodia, joka yrittää saavuttaa tavoitteen.

---

### **5. Esimerkki suoritus**

#### Alkutilanne:
- Energia: 200, Raha: 50.

#### Toimintojen sekvenssi:
1. **Syöminen ("Eat")**:
   - Energia: +200 → 300.
   - Raha: -50 → 0.

2. **Työnteko ("Work")**:
   - Energia: -100 → 150.
   - Raha: +100 → 100.

3. **Työnteko ("Work")**:
   - Energia: -100 → 50.
   - Raha: +100 → 200.

4. **Nukkuminen ("Sleep")** (pakotettu):
   - Energia: +100 → 150.

...jatkuu, kunnes tavoite (rahaa vähintään 1000) saavutetaan.

---

### Lopputulos:
Koodi simuloi realistisesti toimintojen suunnittelua, jossa täytyy tasapainottaa energian, rahan ja hereilläolon välillä. Lopulta se saavuttaa tavoitteensa ilman sääntöjen rikkomista.

In [16]:
class Action:
    def __init__(self, name, energy_change, money_change):
        self.name = name
        self.energy_change = energy_change
        self.money_change = money_change

    def execute(self, state):
        state['energy'] += self.energy_change
        state['money'] += self.money_change
        return state


class Planner:
    def __init__(self, actions, goal, max_consecutive_actions=3, max_energy=300, max_wakefulness=5):
        self.actions = actions
        self.goal = goal
        self.max_consecutive_actions = max_consecutive_actions
        self.max_energy = max_energy
        self.max_wakefulness = max_wakefulness  # Kuinka monta toimintoa voi tehdä ilman nukkumista
        self.wakefulness = 0  # Laskuri hereillä ololle

    def is_valid(self, state, action, action_history):
        # Tarkista, ettei energia mene yli maksimin tai alle 0
        new_energy = state['energy'] + action.energy_change - 50  # Jokainen toiminto kuluttaa energiaa 50
        if new_energy <= 0 or new_energy > self.max_energy:
            return False

        # Tarkista, ettei raha mene alle 0
        if state['money'] + action.money_change <= 0:
            return False

        # Jos hereilläolo ylittää maksimin, vain nukkuminen on sallittua
        if self.wakefulness >= self.max_wakefulness and action.name != "Sleep":
            return False

        # Tarkista, ettei samaa toimintoa tehdä 3 kertaa peräkkäin
        if len(action_history) >= self.max_consecutive_actions - 1:
            if all(a == action.name for a in action_history[-(self.max_consecutive_actions - 1):]):
                return False
        return True

    def plan(self, state):
        action_history = []
        print(f"Goal: Earn at least {self.goal} money while managing energy and avoiding restrictions.")
        print(f"Initial State: {state}\n")
        while state['money'] < self.goal:
            action_chosen = False
            for action in self.actions:
                if self.is_valid(state, action, action_history):
                    # Vähennä energiaa 50 toiminnon yleiskustannuksena
                    state['energy'] -= 50
                    # Suorita toiminto
                    state = action.execute(state)
                    action_history.append(action.name)
                    print(f"Performed: {action.name}, State: {state}")
                    action_chosen = True

                    # Päivitä hereilläolo
                    if action.name == "Sleep":
                        self.wakefulness = 0  # Nollaa hereilläolo laskuri
                    else:
                        self.wakefulness += 1  # Kasvata hereilläoloa
                    break
            if not action_chosen:
                raise ValueError("No valid actions available!")
        print("\nGoal reached!")
        return state, action_history


# Define actions
actions = [
    Action("Eat", energy_change=200, money_change=-50),
    Action("Sleep", energy_change=100, money_change=0),
    Action("Work", energy_change=-100, money_change=100),
]

# Initial state and goal
initial_state = {'energy': 200, 'money': 50}
goal = 1000

# Create planner and plan
planner = Planner(actions, goal, max_wakefulness=5)  # Enintään 5 toimintoa ilman nukkumista
try:
    final_state, action_sequence = planner.plan(initial_state)
    print("\nFinal State:")
    print(final_state)
    print("\nAction Sequence:")
    print(action_sequence)
except ValueError as e:
    print(e)


Goal: Earn at least 1000 money while managing energy and avoiding restrictions.
Initial State: {'energy': 200, 'money': 50}

Performed: Sleep, State: {'energy': 250, 'money': 50}
Performed: Sleep, State: {'energy': 300, 'money': 50}
Performed: Work, State: {'energy': 150, 'money': 150}
Performed: Eat, State: {'energy': 300, 'money': 100}
Performed: Work, State: {'energy': 150, 'money': 200}
Performed: Eat, State: {'energy': 300, 'money': 150}
Performed: Work, State: {'energy': 150, 'money': 250}
Performed: Sleep, State: {'energy': 200, 'money': 250}
Performed: Sleep, State: {'energy': 250, 'money': 250}
Performed: Work, State: {'energy': 100, 'money': 350}
Performed: Eat, State: {'energy': 250, 'money': 300}
Performed: Sleep, State: {'energy': 300, 'money': 300}
Performed: Work, State: {'energy': 150, 'money': 400}
Performed: Eat, State: {'energy': 300, 'money': 350}
Performed: Work, State: {'energy': 150, 'money': 450}
Performed: Eat, State: {'energy': 300, 'money': 400}
Performed: Wo